# Imports

In [57]:
from pathlib import Path
from tqdm.notebook import tqdm_notebook

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from src.metrics.metrics import hierarchical_f1_score
from src.data_loaders.data_loaders import load_data
from src.data_preprocessors.preprocessors import preprocess_data, fill_description_nans

# Load data

In [58]:
data_dir = Path('data/raw')

In [59]:
train_data, test_data, categories_data = load_data(data_dir=data_dir)

In [60]:
train_data

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790
3,1264824,Серьги,None,None,0.000000,0,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401
...,...,...,...,...,...,...,...
283447,584544,Эфирное масло аромамасло 20мл,Аромамаркетинг – это мощный инструмент по созд...,Выберите аромат:Ваниль|Персик|Холл гостиницы|Н...,4.500000,6,2674
283448,1229689,"Форма для выпечки печенья ""Орешки""","Орешки со сгущенкой, форма для приготовления.",None,5.000000,1,13554
283449,904913,Магнит символ Нового года-Тигренок/(по 3 шт в уп),None,None,5.000000,1,11617
283450,1413201,"Рифленный нож / слайсер для фигурной нарезки, ...","Такими ножами удобно резать фрукты, овощи, сыр...","Вид:19,5х6 см",0.000000,0,14030


In [61]:
test_data

,id,title,short_description,name_value_characteristics,rating,feedback_quantity
0,1070974,Браслет из натуральных камней LOTUS,None,None,0.000000,0
1,450413,Fusion Life - Шампунь для сухих и окрашенных в...,None,None,4.333333,6
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",None,3.708333,24
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,None,0.000000,0
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акр...",Стильная и яркая брошь ручной работы! Великоле...,None,0.000000,0
...,...,...,...,...,...,...
70859,967535,Носки с мехом куницы авокадо разноцветные,Пуховые носки с мехомом куницы с авакадо.,None,5.000000,3
70860,1488636,"Эфирное масло Сосны, 10 мл, от КедрМаркет","Масло сосны повышает защитную функцию кожи, уп...",None,0.000000,0
70861,827510,Компект (футболка+шорты),"Отличный комплект. Удобный, комфортный.",None,0.000000,0
70862,529244,Купальный костюм Mark Formelle,None,Российский размер:40|42|44|46,0.000000,0


In [62]:
categories_data

,id,title,parent_id
0,1,Все категории,0
1,114,Урбеч,1913
2,115,Варенье и джемы,328
3,128,Сухие завтраки,2475
4,131,Масла,2475
...,...,...,...
3365,14555,Насадки и запчасти,11691
3366,14556,Швейные машины,10062
3367,14557,Матрасы,2894
3368,14558,Ледянки и тюбинги,10092


# Some EDA

В решении буду использовать только *title* + *short_description*, поэтому лишние признаки можно удалить

In [63]:
target = 'category_id'

In [64]:
train_data.category_id.value_counts().tail()

13756    2
13007    2
2598     2
11917    2
13787    2
Name: category_id, dtype: int64

Видим, что есть какое-то количество категорий с очень маленьким количеством товаров в них, для обучения лучше будет заменить эти категории на родительские, чтобы увеличить количество товаров в категории, т.к. метрика учитывает не только конечную категорию, а все дерево. Таким образом мы сильно снизим количество классов для классификатора и улучшим качество. 

Выделим категории, в которых меньше 100 товаров

In [65]:
train_data['new_category'] = train_data['category_id']

In [66]:
def get_categories_to_change(data: pd.DataFrame, threshold: int = 100) -> list[int]:
    products_per_categories = data.groupby('new_category', as_index=False) \
                                  .agg(unique_products=('id', 'count')).sort_values(by='unique_products')

    return products_per_categories[products_per_categories.unique_products < threshold].new_category.tolist()

In [67]:
print(f'Количество категорий, в которых менее 100 товаров: {len(get_categories_to_change(train_data))}')

Количество категорий, в которых менее 100 товаров: 750


Таких категорий оказалось почти 750, а это ~3/4 от 1200, посмотрим, что получится, если теперь заменить эти категории на родительские

In [68]:
def change_category_to_parent(categories_to_change: list[int],
                              data: pd.DataFrame,
                              categories_data: pd.DataFrame) -> pd.DataFrame:
    category_to_parent_dict = dict(categories_data.loc[categories_data.id.isin(categories_to_change), ['id', 'parent_id']].values)
    data.loc[data.new_category.isin(categories_to_change), 'new_category'] = data.new_category.map(category_to_parent_dict)
    data['new_category'] = data['new_category'].astype(int)

    return data

def show_products_per_categories(data: pd.DataFrame, threshold: int = 100):
    products_per_categories = data.groupby('new_category', as_index=False) \
                                  .agg(unique_products=('id', 'count')) \
                                  .sort_values(by='unique_products', ascending=False)
    return products_per_categories[products_per_categories.unique_products < threshold]

In [69]:
while len(get_categories_to_change(train_data)) > 10:
    categories_to_change = get_categories_to_change(train_data)
    train_data = change_category_to_parent(categories_to_change=categories_to_change,
                                           data=train_data,
                                           categories_data=categories_data)

In [70]:
show_products_per_categories(train_data)

,new_category,unique_products
52,10018,94
76,10118,70


In [71]:
train_data.new_category.nunique()

595

Теперь имеем 595 категорий, в которых приемлемое количество товаров, удалим из данных товары, у которых категория стала равна 1, т.к. это категория "Все категории"

In [72]:
train_data = train_data[train_data.new_category != 1]

In [73]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283278 entries, 0 to 283451
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          283278 non-null  int64  
 1   title                       283278 non-null  object 
 2   short_description           133058 non-null  object 
 3   name_value_characteristics  50352 non-null   object 
 4   rating                      283278 non-null  float64
 5   feedback_quantity           283278 non-null  int64  
 6   category_id                 283278 non-null  int64  
 7   new_category                283278 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 19.5+ MB


Видим большое количество пропусков в колонке с описанием

In [74]:
train_data.short_description.value_counts()

                                                                                                                                      2782
None                                                                                                                                  1691
Тачскрином называют стекло, отвечающее за прикосновения на телефоне, это самая хрупкая часть устройства.                               399
Коллекция TNL 8 чувств - это богатая палитра самых ярких, смелых оттенков в флаконах с пудровым напылением «soft-touch».               237
Пульт отличается качественной сборкой сертифицированного предприятия Huayu                                                             228
                                                                                                                                      ... 
Заколка-зажим                                                                                                                            1
Колготки женские сетчатые и

Заодно видим, что есть большое количество строк с пустыми описаниями ('') и с одинаковыми описаниями, т.е. в данных большое количество дубликатов

In [75]:
train_data = fill_description_nans(train_data)
test_data = fill_description_nans(test_data)

/home/ilya/Тестовые/kazan-express/src/data_preprocessors/preprocessors.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['short_description'] = data['short_description'].fillna(fill_value)


## Find duplicates

In [76]:
duplicates_data = train_data.groupby(['title', 'short_description'], as_index=False) \
                            .agg(unique_id_number=('id', 'count')) \
                            .sort_values(by='unique_id_number', ascending=False)
duplicates_data[duplicates_data.unique_id_number > 2]

,title,short_description,unique_id_number
222847,Футболка ТВОЕ,описания нет,480
185786,Серьги,описания нет,254
28132,Брюки Koton,описания нет,217
200879,Сумка женская,описания нет,199
197454,Спортивные брюки ТВОЕ,описания нет,181
...,...,...,...
49430,Джемпер вискоза,описания нет,3
136130,Носки мужские короткие однотонные,описания нет,3
80367,Колготки ажурные Peppy Woolton,описания нет,3
46204,Держатель для фена самоклеящийся,Держатель для фена самоклеящийся,3


In [77]:
print(f'Уникальных названий товаров: {train_data.title.nunique()}')
print(f'Уникальный id товаров: {train_data.id.nunique()}')

Уникальных названий товаров: 239851
Уникальный id товаров: 283278


Почти 4000 товаров являются дубликатами, всего же уникальных товаров 240000; оставим только один экземпляр каждого продублированного товара

In [78]:
train_data = train_data.drop_duplicates(subset=['title', 'short_description', 'category_id'])
train_data

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,new_category
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",описания нет,None,4.666667,9,13408,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790,11790
3,1264824,Серьги,описания нет,None,0.000000,0,14076,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401,12401
...,...,...,...,...,...,...,...,...
283447,584544,Эфирное масло аромамасло 20мл,Аромамаркетинг – это мощный инструмент по созд...,Выберите аромат:Ваниль|Персик|Холл гостиницы|Н...,4.500000,6,2674,2674
283448,1229689,"Форма для выпечки печенья ""Орешки""","Орешки со сгущенкой, форма для приготовления.",None,5.000000,1,13554,13554
283449,904913,Магнит символ Нового года-Тигренок/(по 3 шт в уп),описания нет,None,5.000000,1,11617,11617
283450,1413201,"Рифленный нож / слайсер для фигурной нарезки, ...","Такими ножами удобно резать фрукты, овощи, сыр...","Вид:19,5х6 см",0.000000,0,14030,14030


## Preprocess data
1) Привести все text_features к нижнему регистру,
2) Соединить title и description,
3) Извлечь признаки из текстовых фичей при помощи CountVectorizer
4) Разбить данные на тренировочную и валидационную выборки


In [79]:
def build_full_path_str(category_id):
    labels_path = _build_full_path(category_id, type_='str')

    return " ".join(labels_path[::-1])

def build_full_path_id(category_id):
    ids_path = _build_full_path(category_id, type_='int')
    return ids_path

def _build_full_path(category_id, type_: str = 'int'):
    labels_path = []
    id_path = []
    leaf_category = category_id

    exit_flag = False
    while not exit_flag:
        try:
            parent = categories_data.loc[categories_data.id == leaf_category, 'parent_id'].values[0]
        except IndexError:
            exit_flag = True
            continue

        label = categories_data.loc[categories_data.id == leaf_category, 'title'].values[0]
        labels_path.append(label)
        id_path.append(leaf_category)

        if parent == 0 or parent == 1:
            exit_flag = True
        else:
            leaf_category = parent
    if type_ == 'str':
        return labels_path
    elif type_ == 'int':    
        return id_path 


used_categories = train_data.new_category.unique()
categories_data['full_path'] = ''
categories_data['full_path_id'] = ''
categories_data['full_path_id'] = categories_data['full_path_id'].astype(object)
for row_index in tqdm_notebook(categories_data.index):
    cat_id = categories_data.loc[row_index, 'id']
    if cat_id in used_categories:
        full_path = build_full_path_str(cat_id)
        full_path_id = build_full_path_id(cat_id)
        categories_data.loc[row_index, 'full_path'] = full_path
        categories_data.at[row_index, 'full_path_id'] = full_path_id

categories_data['full_path'] = categories_data['full_path'].str.lower()
categories_data[categories_data.full_path != ''].sample(10)

  0%|          | 0/3370 [00:00<?, ?it/s]

,id,title,parent_id,full_path,full_path_id
2332,13277,Свадебные товары,10118,товары для дома товары для праздников свадебны...,"[13277, 10118, 10018]"
1742,12604,Воздушные шары и аксессуары,10934,товары для дома товары для праздников оформлен...,"[12604, 10934, 10118, 10018]"
1643,12493,Футболки,11306,одежда одежда для девочек футболки и майки фут...,"[12493, 11306, 10221, 10014]"
2810,13816,Патчи,10137,красота уход за лицом патчи,"[13816, 10137, 10012]"
186,2740,Полки и подставки,12842,товары для дома хозяйственные товары аксессуар...,"[2740, 12842, 10110, 10018]"
2406,13362,Коннекторы,10073,электроника аксессуары для электроники коннекторы,"[13362, 10073, 10020]"
1562,12407,Термокружки,10586,товары для дома товары для кухни термосы и тер...,"[12407, 10586, 10115, 10018]"
2552,13527,Обложки для документов,10143,"аксессуары женские аксессуары кошельки, ключни...","[13527, 10143, 10023, 10003]"
1783,12648,Парфюмерная вода,10084,красота парфюмерия парфюмерная вода,"[12648, 10084, 10012]"
1570,12415,Боди и комбинезоны,10095,одежда одежда для новорождённых боди и комбине...,"[12415, 10095, 10014]"


In [80]:
categories_data['title'] = categories_data['title'].str.lower()
categories_data.head()

,id,title,parent_id,full_path,full_path_id
0,1,все категории,0,,
1,114,урбеч,1913,,
2,115,варенье и джемы,328,,
3,128,сухие завтраки,2475,,
4,131,масла,2475,,


In [81]:
drop_columns = ['short_description', 'title', 'rating', 'feedback_quantity', 'name_value_characteristics']
train_data = preprocess_data(train_data, drop_columns)
test_data = preprocess_data(test_data, drop_columns)

In [82]:
train_data.head()

,id,category_id,new_category,description
0,1267423,2693,2693,muhle manikure песочные колпачки для педикюра ...
1,128833,13408,13408,"sony xperia l1 защитное стекло 2,5d описания нет"
2,569924,11790,11790,"конверт для денег прекрасная роза, 16,5 х 8 см..."
3,1264824,14076,14076,серьги описания нет
4,1339052,12401,12401,наклейки на унитаз для туалета на крышку бачок...


In [83]:
new_categories_data = categories_data[categories_data.id.isin(train_data.new_category.unique().tolist())]
new_categories_data

,id,title,parent_id,full_path,full_path_id
65,2599,купальники,11351,одежда женская одежда белье и купальники купал...,"[2599, 11351, 10116, 10014]"
67,2601,майки и топы бельевые,11351,одежда женская одежда белье и купальники майки...,"[2601, 11351, 10116, 10014]"
68,2602,корректирующее белье,11351,одежда женская одежда белье и купальники корре...,"[2602, 11351, 10116, 10014]"
71,2605,колготки,10416,одежда одежда для мальчиков носки колготки,"[2605, 10416, 10222, 10014]"
73,2607,часы,2606,аксессуары женские аксессуары часы и ремешки часы,"[2607, 2606, 10023, 10003]"
...,...,...,...,...,...
3346,14503,стекла,12899,электроника умные часы и фитнес браслеты защит...,"[14503, 12899, 10141, 10020]"
3349,14538,запчасти для планшетов и электронных книг,10487,"электроника ноутбуки, планшеты и электронные к...","[14538, 10487, 10030, 10020]"
3350,14539,радиаторы и кулеры,13073,электроника компьютерная техника комплектующие...,"[14539, 13073, 10641, 10074, 10020]"
3356,14546,домашние платья и туники,11116,одежда женская одежда домашняя одежда домашние...,"[14546, 11116, 10116, 10014]"


In [84]:
vectorizer = CountVectorizer(ngram_range=(1,1), binary =True, max_features = 5000)

In [85]:
vectorizer.fit(train_data.description.values.tolist() + test_data.description.values.tolist())

CountVectorizer(binary=True, max_features=5000)

In [86]:
train_features = vectorizer.transform(train_data.description.values)
test_features = vectorizer.transform(test_data.description.values)

In [87]:
labels_data = train_data[['category_id', 'new_category']]

x_train, x_val, y_train, y_val = train_test_split(train_features, 
                                                    labels_data,
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=100)

# Model

In [70]:
%%time
model = LogisticRegression(C=1.0)
model.fit(x_train, y_train[['new_category']])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 9min 5s, sys: 3min 19s, total: 12min 25s
Wall time: 11min 59s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [71]:
predicts = model.predict(x_val)

In [90]:
decoder = dict(categories_data.loc[categories_data.full_path_id != '', ['id', 'full_path_id']].values)

In [91]:
decoder

{2599: [2599, 11351, 10116, 10014],
 2601: [2601, 11351, 10116, 10014],
 2602: [2602, 11351, 10116, 10014],
 2605: [2605, 10416, 10222, 10014],
 2607: [2607, 2606, 10023, 10003],
 2610: [2610, 2609, 10021, 10003],
 2631: [2631, 11328, 10091, 10012],
 2632: [2632, 11328, 10091, 10012],
 2634: [2634, 10648, 10137, 10012],
 2635: [2635, 10648, 10137, 10012],
 2636: [2636, 10648, 10137, 10012],
 2662: [2662, 10476, 10137, 10012],
 2663: [2663, 10137, 10012],
 2674: [2674, 2673, 10012],
 2677: [2677, 13817, 10169, 10012],
 2690: [2690, 10163, 10012],
 2691: [2691, 10113, 10012],
 2692: [2692, 10113, 10012],
 2693: [2693, 10355, 10113, 10012],
 2725: [2725, 10282, 10115, 10018],
 2728: [2728, 10282, 10115, 10018],
 2729: [2729, 10890, 10115, 10018],
 2730: [2730, 10191, 10018],
 2733: [2733, 12823, 10110, 10018],
 2737: [2737, 12842, 10110, 10018],
 2739: [2739, 12842, 10110, 10018],
 2740: [2740, 12842, 10110, 10018],
 2741: [2741, 12842, 10110, 10018],
 2742: [2742, 12842, 10110, 10018],
 

In [73]:
predicts_paths = [decoder[pred] for pred in predicts]

In [74]:
true_paths = [decoder[pred] for pred in y_val.category_id.values]

In [75]:
hierarchical_f1_score(predicts=predicts_paths, 
                      true=true_paths)

0.8704524197643034

In [76]:
train_preds = model.predict(x_train)

In [77]:
train_predicts_paths = [decoder[pred] for pred in train_preds]
true_predicts_paths = [decoder[pred] for pred in y_train.category_id.values]

In [78]:
hierarchical_f1_score(predicts=train_predicts_paths, 
                      true=true_predicts_paths)

0.9118014492820106

In [79]:
predicts_final = model.predict(test_features)

In [80]:
predicts_final

array([11574, 11878, 13299, ..., 14171, 13069, 14034])

In [82]:
test_data['predicted_category_id'] = predicts_final
test_data

,id,description,predict,predicted_category_id
0,1070974,браслет из натуральных камней lotus описания нет,11574,11574
1,450413,fusion life - шампунь для сухих и окрашенных в...,11878,11878
2,126857,"микрофон для пк jack 3,5мм всенаправленный уни...",13299,13299
3,1577569,серьги гвоздики сердце серьги гвоздики сердце,13061,13061
4,869328,"чёрно-красная стильная брошь ""тюльпаны"" из акр...",12813,12813
...,...,...,...,...
70859,967535,носки с мехом куницы авокадо разноцветные пухо...,13143,13143
70860,1488636,"эфирное масло сосны, 10 мл, от кедрмаркет масл...",2674,2674
70861,827510,компект (футболка+шорты) отличный комплект. уд...,14171,14171
70862,529244,купальный костюм mark formelle описания нет,13069,13069


In [83]:
submission = test_data[['id', 'predicted_category_id']]
submission

,id,predicted_category_id
0,1070974,11574
1,450413,11878
2,126857,13299
3,1577569,13061
4,869328,12813
...,...,...
70859,967535,13143
70860,1488636,2674
70861,827510,14171
70862,529244,13069


In [86]:
submission.to_parquet(data_dir / 'result.parquet')